In [2]:
from pandas import read_stata
import pandas as pd
from lcsscaseapi.client import LCSSClient
from lcsscaseapi.types import USCircuitCaseMeta, USJudge, JudgeRuling
import datetime
import os
from dotenv import load_dotenv

In [3]:
df = read_stata("BloombergCASELEVEL_Touse.dta")

print(len(df["caseid"]))

387898


In [48]:
#df = df.head(n=1000) # sub sample to play with

In [49]:
def create_case_meta(row):
    case_id = row['caseid']
    circuit_num = row['Circuit']
    circuit_name = USCircuitCaseMeta.CIRCUITS[circuit_num]
    self_cite = row['citation']
    docket_number = row['docketnumber']
    if not pd.isnull(row['date']):
        date = datetime.date(int(row['year']), int(row['month']), int(row['day']))
    else:
        date = None
    tags = construct_tags(row)
    outcome = construct_outcome(row)

    return USCircuitCaseMeta(case_id=case_id, circuit_name=circuit_name, self_cite=self_cite, 
                                docket_number=docket_number, date=date, tags=tags, outcome = outcome)

def construct_tags(row):
    tags = []
    if row['Criminal'] == 1:
        tags.append('CRIMINAL')
    
    if row['Civil_Rights'] == 1:
        tags.append('CIVIL RIGHTS')

    if row['First_Amendment'] == 1:
        tags.append('FIRST AMENDMENT')
    
    if row['Due_Process'] == 1:
        tags.append('DUE PROCESS')
    
    if row['Privacy'] == 1:
        tags.append('PRIVACY')
    
    if row['Labor_Relations'] == 1:
        tags.append('LABOR RELATIONS')
    
    if row['Econ_Activity'] == 1:
        tags.append('ECONOMIC ACTIVITY')

    if row['Miscellanous'] == 1:
        tags.append('MISCELLANEOUS')

    return tags

def construct_outcome(row):
    outcomes = []

    if row["Affirmed"] == 1:
        outcomes.append("AFFIRMED")
    
    if row["AffirmedInPart"] == 1:
        outcomes.append("AFFIRMED (IN PART)")
    
    if row["Reversed"] == 1:
        outcomes.append("REVERSED")

    if row["ReversedInPart"] == 1:
        outcomes.append("REVERSED (IN PART)")
    
    if row["Vacated"] == 1:
        outcomes.append("VACATED")

    if row["VacatedInPart"] == 1:
        outcomes.append("VACATED (IN PART)")

    if row["Remanded"] == 1:
        outcomes.append("REMANDED")

    if len(outcomes) == 0:
        return None
    else:
        return ",".join(outcomes)



    

In [50]:
# takes 39.5s to run
cases = df.apply(create_case_meta, axis=1)

print(len(cases))

1000


In [4]:
load_dotenv()

USERNAME = os.getenv('ACCOUNT')
PWD = os.getenv('PASSWORD')

client = LCSSClient(username=USERNAME, password=PWD)

In [78]:
#returned_cases = client.upload_us_cases(cases)

In [5]:
# Now, time to create all the USJudges and JudgeRulings

# The idea will be to create judges from every single case
# Then later collapse this into just the unique judges (which match on every field, since its unclear if orig_name's are unique)

# Given a row in the stata file, creates a tuple of USJudges
def create_judge_tuple(row):
    return (create_judge(row, 1), create_judge(row, 2), create_judge(row, 3))

# For a given row and judge number, returns that judge's details as a USJudge object
# judgenum = 1, 2 or 3 for judges labelled j1, j2 or j3
def create_judge(row, judgenum):
    name = judge_property(row, judgenum, "name")  # if empty string, return None is what this does
    name = None if name == "" else name
    orig_name = judge_property(row, judgenum, "Origname")
    name = orig_name if name == None else name # if name is missing replace with the Origname
    orig_name = None # otherwise, remove the orig_name, it no longer bears any info and the same person can have two orig names
    gender_num = judge_property(row, judgenum, "gender")
    gender = None if pd.isnull(gender_num) else USJudge.GENDERS[int(gender_num)-1] # 1 is converted to MALE, 2 is converted to FEMALE
    party_num = judge_property(row, judgenum, "party")
    party_num = party_num_cleaning(party_num=party_num, name=name)
    party = None if pd.isnull(party_num) else USJudge.PARTIES[1-int(party_num)] # 1 is converted to Democrat, 0 is converted to Republican
    senior_num = judge_property(row, judgenum, "Senior")
    senior = None if pd.isnull(senior_num) else bool(senior_num)

    assert name != ""
    assert orig_name != ""
    assert gender == USJudge.MALE or gender_num != 1
    assert party == USJudge.DEMOCRAT or party_num != 1
    assert senior != False or senior_num == 0

    return USJudge(name=name, orig_name=orig_name, gender=gender, senior=senior, party=party)

# For a given judge and property, returns the property
# For example, calling judge_property(row, "j1", "name") will fetch j1name from the row
def judge_property(row, judgenum, judgeprop):
    return row["j" + str(judgenum) + judgeprop]

# Convert unclean party numbers to 1 for democrat, 0 for republican
def party_num_cleaning(party_num, name):
    party_num = 0 if name == "BOND, HUGH LENNOX" else party_num # BOND Appointed by Ulysses S Grant (R)
    party_num = 1 if name == "HAYS, PAUL" else party_num # HAYS Appointed by John F Kennedy (D)
    party_num = 0 if name == "MAHONEY, J. DANIEL" else party_num # MAHONEY Appointed by Ronald Reagan (R)
    party_num = 0 if name == "BURNS, LOUIS HENRY" else party_num # BURNS Appointed by Calvin Coolidge (R)
    party_num = 1 if name == "BAER, HAROLD, JR." else party_num # many Orignames Appointed by Bill Clinton (D) - also a district court judge

    party_num = None if party_num == 3 else party_num # unclear what party number of 3 really means, leave it blank for now

    return party_num


In [6]:
# Case X3ACOQ is problematic (974 F2d 1200), row 79377 (with 1-index, so 79376 with 0-index)
# Has an identical judge,  NELSON, DOROTHY WRIGHT, but with two different orig_names
# NELSON and D. W. NELSON
# It seems like these are supposed to be two different judges (the j2name field is wrongly populated!)
# See https://law.justia.com/cases/federal/appellate-courts/F2/974/1200/437757/
# But its not clear who NELSON (j2) is supposed to be otherwise (many possible NELSONs)
# From tabbing the names and circuits, it seems lke there's two ninth circuit judges with surname NELSON
# Thomas G Nelson (NELSON, THOMAS G.) who served from 1990 to 2004 on the 9th circuit
# NELSON, THOMAS LEVERETT who served in the district court from 1879 to 1897
# So its probably the first guy
# So, let's fix this

# first make sure we have the right row
assert df.iloc[79376].caseid == "X3ACOQ"
assert df.iloc[79376].citation == "974 F2d 1200"
assert df.iloc[79376].j2name == "NELSON, DOROTHY WRIGHT"
assert df.iloc[79376].j2Origname == "NELSON"
df.at[79376, 'j2name'] = "NELSON, THOMAS G."
df.at[79376, 'j2gender'] = 1
df.at[79376, 'j2party'] = 0 # He was a Republican Male
df.at[79376, 'j2Senior'] = 0 # He was not senior in this 1992 case, only from 2004
df.at[79376, 'Author'] = "D. W. NELSON" # Dorothy White authored this, not Thomas

assert df.iloc[79376].j2name == "NELSON, THOMAS G." # make sure it set correctly
#df.iloc[79376].j2name = "NELSON, THOMAS G."

In [7]:
judges_per_case = df.apply(create_judge_tuple, axis=1)

print(len(judges_per_case))

387898


In [8]:
# Extract unique judges
all_judges = set()
for judges in judges_per_case:
    (j1, j2, j3) = judges
    #assert j1.judge_orig_name != None
    #assert j2.judge_orig_name != None
    #assert j3.judge_orig_name != None

    all_judges.add(j1)
    all_judges.add(j2)
    all_judges.add(j3)

print(len(all_judges))

4502


In [10]:
judge_list = list(all_judges)

uploaded_judges = client.upload_us_judges(judge_list)

#uploaded_judges = client.get_us_judges() # if running the remaining code after the upload, fetch the uploaded judges again - need their IDs

In [11]:
print(len(uploaded_judges))


4502


In [12]:
from copy import deepcopy
# Create dictionary mapping id-less judge to ID
judge_id_dict = dict()
for judge in uploaded_judges:
    idless_judge = deepcopy(judge)
    idless_judge.id = None
    assert judge.id != None
    judge_id_dict[idless_judge] = judge.id

In [13]:
# create a version of judges_per_case but each judge now has the ID
def assign_id(judge):
    return_judge = deepcopy(judge)
    return_judge.id = judge_id_dict[judge]
    return return_judge
    
judges_per_case_with_id =  [(assign_id(j1), assign_id(j2), assign_id(j3)) for j1,j2,j3 in judges_per_case]

In [14]:
df['judges_per_case'] = judges_per_case_with_id # add a column for the tuple of judge objects

In [15]:
def repopulate_orig_name(row):
    # orig names are used in matching judges to whether they were dissenters/concurrers/authors on a case
    (j1, j2, j3) = row['judges_per_case']
    j1 = deepcopy(j1)
    j2 = deepcopy(j2)
    j3 = deepcopy(j3)

    j1.judge_orig_name = judge_property(row, 1, "Origname")
    j2.judge_orig_name = judge_property(row, 2, "Origname")
    j3.judge_orig_name = judge_property(row, 3, "Origname")

    return (j1, j2, j3)

judges_per_case_complete = df.apply(repopulate_orig_name, axis=1)

In [16]:
print(len(judges_per_case_complete))
df['judges_per_case'] = judges_per_case_complete

387898


In [17]:
# THIS ENTIRE SECTION IS EXPLORATORY
# To see how to handle problems of unclean names
# not actually used to execute any action needed to upload judge rulings

dissenter_collision = 0
dissenter_not_found = 0
concurrer_collision = 0
concurrer_not_found = 0
author_collision = 0
author_not_found = 0

author_collision_list = []
def check_judge_matches(row):
    dissenters = row['JudgeDissentingTouse'].split(sep = "|")
    dissenters = [dissenter.strip().upper()  for dissenter in dissenters if dissenter.strip() != ""]
    concurrers = row['JudgeconcurringTouse'].split(sep = "|")
    concurrers = [concurrer.strip().upper() for concurrer in concurrers if concurrer.strip() != ""]
    author = row['Author']

    (j1, j2, j3) = row['judges_per_case']
    for dissenter in dissenters:
        res = matches_one_judge(j1, j2, j3, dissenter, "dissenter", row['caseid'])
        if res != True:
            if res=="COLLISION":
                global dissenter_collision
                print("Dissenter collision", row['caseid'], row['citation'], dissenter, row['judges_per_case'])
                dissenter_collision = dissenter_collision + 1
            elif res=="NOT FOUND":
                global dissenter_not_found
                dissenter_not_found = dissenter_not_found + 1
    for concurrer in concurrers:
        res = matches_one_judge(j1, j2, j3, concurrer, "concurrer", row['caseid'])
        if res != True:
            if res=="COLLISION":
                global concurrer_collision
                print("Concurrer collision", row['caseid'], row['citation'],concurrer, row['judges_per_case'])
                concurrer_collision = concurrer_collision + 1
            elif res=="NOT FOUND":
                global concurrer_not_found
                concurrer_not_found = concurrer_not_found + 1
    if not pd.isnull(author) and author.strip() != "" and author != "PER CURIAM":
        res = matches_one_judge(j1, j2, j3, author, "author", row['caseid'])
        if res != True:
            if res=="COLLISION":
                global author_collision
                author_collision = author_collision + 1
                author_collision_list.append((row['caseid'], author, j1, j2, j3))
            elif res=="NOT FOUND":
                global author_not_found
                author_not_found = author_not_found + 1

def matches_one_judge(j1, j2, j3, name, purpose, caseid):
    # this function is to check that any named dissenter, concurrer or author (purpose must be one of these 3)
    # corresponds to exactly one judge
    # IE isn't typo'ed and corresponds to no judge
    # isn't confusingly possibly two different judges

    # Tries to match by name and by orig name
    res = clean_matches(all_matching_judges_to_name([j1, j2, j3], name), name, purpose, caseid)
    if len(res) == 1:
        return True
    elif len(res) == 0:
        return "NOT FOUND"
    
    return "COLLISION"

def remove_duplicates(matches):
    match_set = set()
    for match in matches:
        match_set.add(match)
    matches = list(match_set) # removes duplicates
    return matches

def clean_matches(matches, name, purpose, caseid):
    # Some cleaning for some unruly match cases
    # first remove duplicates
    matches = remove_duplicates(matches)
    # then fix the WILL HUBERT L case (881 F2d 1128)
    if name == "WILL" and purpose == "concurrer" and caseid == "X3A693":
        correct_match = None
        for match in matches:
            if match.judge_name == "WILL, HUBERT L.":
                correct_match = match
        matches = [correct_match]
    
    assert matches != None # just a sanity check
    return matches

def all_matching_judges_to_name(judges, name):
    # Try first to match by whether the name is in the judge's full name
    # if this only matches one judge, conclude that's the correct one
    in_judge_name = [judge for judge in judges if name in judge.judge_name]
    if len(in_judge_name) == 1:
        return in_judge_name
    
    # If this isn't true, try to check if the name is in the judge's orig_name
    in_orig_name = [judge for judge in judges if name in judge.judge_orig_name]
    if len(in_orig_name) == 1:
        return in_orig_name
    if len(in_orig_name) > 1:
        # if the name is in two judge's orig_names (e.g. WLLIAM matches with j1: WILLIAM and j2: WILLIAMSON)
        # search for an exact match
        eq_orig_name = [judge for judge in judges if name == judge.judge_orig_name]
        if len(eq_orig_name) >= 1:
            # return the list with the least found judges, but favor a list that finds any judges over one that finds none
            # ie, collision before claiming not-found
            if len(in_judge_name) < len(eq_orig_name) and len(in_judge_name) != 0:
                return in_judge_name
            else:
                return eq_orig_name
        else:
            return in_orig_name
    # if you reach here, in_orig_name is empty
    # so return in_judge_name; if its empty or has more than one, its the better answer
    return in_judge_name


df.apply(check_judge_matches, axis = 1)
print("Dissenter collison", dissenter_collision)
print("Dissenter not found", dissenter_not_found)
print("Concurrer collison", concurrer_collision)
print("Concurrer not found", concurrer_not_found)
print("Author collison", author_collision)
print("Author not found", author_not_found)

Dissenter collison 0
Dissenter not found 2451
Concurrer collison 0
Concurrer not found 1507
Author collison 32
Author not found 11485


In [18]:
problematic_author_names = set()
problematic_caseids = list()
for author_coll in author_collision_list:
    (caseid, name, _, _, _) = author_coll
    problematic_author_names.add(name)
    problematic_caseids.append(caseid)

print(len(problematic_author_names))
print(problematic_author_names)
#print(author_collision_list)
(df[df.caseid.isin(problematic_caseids)])[["caseid", "citation","Circuit","j1name", "j2name", "j3name", "Author"]]

4
{'ANDERSON', 'WOOD', 'CARNES', 'NEWMAN'}


,caseid,citation,Circuit,j1name,j2name,j3name,Author
45825,X2AI1L,261 F3d 701,7,"WOOD, HARLINGTON, JR.","WILLIAMS, ANN CLAIRE","WOOD, DIANE PAMELA",WOOD
56276,X315UN,142 F3d 999,7,"WOOD, HARLINGTON, JR.","WOOD, DIANE PAMELA","RIPPLE, KENNETH F.",WOOD
60725,X32MDE,966 F2d 1326,10,"ANDERSON, STEPHEN H.","ANDERSON, ALDON J.","EBEL, DAVID M.",ANDERSON
63330,X353EH,96 F3d 1033,7,"WOOD, HARLINGTON, JR.","WOOD, DIANE PAMELA","ESCHBACH, JESSE E.",WOOD
63432,X353MP,96 F3d 260,7,"WOOD, HARLINGTON, JR.","WOOD, DIANE PAMELA","ESCHBACH, JESSE E.",WOOD
70085,X37P0T,149 F3d 603,7,"WOOD, HARLINGTON, JR.","WOOD, DIANE PAMELA","KANNE, MICHAEL S.",WOOD
95892,X3FB17,207 F3d 994,7,"WOOD, HARLINGTON, JR.","WOOD, DIANE PAMELA","KANNE, MICHAEL S.",WOOD
98225,X3FLOD,227 F3d 817,7,"WOOD, HARLINGTON, JR.","WOOD, DIANE PAMELA","KANNE, MICHAEL S.",WOOD
98704,X3FNL0,232 F3d 556,7,"WOOD, HARLINGTON, JR.","WOOD, DIANE PAMELA","KANNE, MICHAEL S.",WOOD
98713,X3FNLH,232 F3d 595,7,"WOOD, HARLINGTON, JR.","WOOD, DIANE PAMELA","ROVNER, ILANA DIAMOND",WOOD


In [19]:
import random

# Finally, actually create the judge_rulings
no_match_count = 0

def create_judge_rulings_tuple(row):
    dissenters = row['JudgeDissentingTouse'].split(sep = "|")
    dissenters = [dissenter.strip().upper()  for dissenter in dissenters if dissenter.strip() != ""]
    concurrers = row['JudgeconcurringTouse'].split(sep = "|")
    concurrers = [concurrer.strip().upper() for concurrer in concurrers if concurrer.strip() != ""]
    author = row['Author']
    (j1, j2, j3) = row['judges_per_case']
    caseid = row['caseid']

    concurring_judges = get_matches(judges=[j1, j2, j3], arr_names=concurrers, purpose="concurrer", caseid=caseid)
    dissenting_judges = get_matches(judges=[j1, j2, j3], arr_names=dissenters, purpose="dissenter", caseid=caseid)
    author_judges = []
    if not pd.isnull(author) and author.strip() != "" and author != "PER CURIAM":
        author_judges = get_matches(judges=[j1, j2, j3], arr_names=[author], purpose="author", caseid=caseid)
    elif author=="PER CURIAM":
        author_judges = [j1, j2, j3]
    
    j1ruling = create_judge_ruling(j1, caseid, j1 in author_judges, j1 in concurring_judges, j1 in dissenting_judges)
    j2ruling = create_judge_ruling(j2, caseid, j2 in author_judges, j2 in concurring_judges, j2 in dissenting_judges)
    j3ruling = create_judge_ruling(j3, caseid, j3 in author_judges, j3 in concurring_judges, j3 in dissenting_judges)

    return (j1ruling, j2ruling, j3ruling)

def get_matches(judges, arr_names, purpose, caseid):
    matches = []
    for name in arr_names:
        matchset = clean_matches(all_matching_judges_to_name(judges, name), name, purpose, caseid)
        if matchset != []:
            # some names just can't be matched, small
            match = random.choice(matchset) # if more than one match, pick one randomly, see README
            matches.append(match)
        else:
            global no_match_count
            no_match_count = no_match_count + 1
    return matches

def create_judge_ruling(judge, case_id, author, concurrer, dissenter):
    vote = JudgeRuling.CONCURRING # assumed to be concurring unless theyre explicitly labelled as a dissenter
    if concurrer and dissenter:
        vote = JudgeRuling.PARTIAL
    elif dissenter:
        vote = JudgeRuling.DISSENTING

    return JudgeRuling(case_id=case_id, judge_id=judge.id, author=author, vote=vote)

rulings_per_case = df.apply(create_judge_rulings_tuple, axis=1)
print(len(rulings_per_case))
print(no_match_count)

387898
15443


In [20]:
print(rulings_per_case[79376])

(JudgeRuling Object: {"author": false, "case": "X3ACOQ", "id": null, "judge": 18265, "vote": "Concurring"}, JudgeRuling Object: {"author": false, "case": "X3ACOQ", "id": null, "judge": 18462, "vote": "Concurring"}, JudgeRuling Object: {"author": true, "case": "X3ACOQ", "id": null, "judge": 17006, "vote": "Concurring"})


In [21]:
# Extract unique JudgeRulings
# sometimes the same judge shows up twice on the same case in the stata
# this should only be one ruling
all_rulings = set()
for rulings in rulings_per_case:
    (jr1, jr2, jr3) = rulings

    all_rulings.add(jr1)
    all_rulings.add(jr2)
    all_rulings.add(jr3)

print(len(all_rulings))

1163663


In [22]:
# check uniqueness of judge id and case_id

uniques = set()

for ruling in list(all_rulings):
    if (ruling.case, ruling.judge) in uniques:
        print(ruling.case, ruling.judge)
        assert False
    uniques.add((ruling.case, ruling.judge))

print(len(uniques))

1163663


In [23]:
# Upload all the judge rulings -- and we're done!
print(len(all_rulings))
client.upload_judge_ruling(list(all_rulings))

1163663


[JudgeRuling Object: {"author": true, "case": "XABFTL", "id": 393062, "judge": 16359, "vote": "Concurring"},
 JudgeRuling Object: {"author": false, "case": "X2NA65", "id": 393063, "judge": 18558, "vote": "Concurring"},
 JudgeRuling Object: {"author": false, "case": "X3M7AT", "id": 393064, "judge": 20141, "vote": "Concurring"},
 JudgeRuling Object: {"author": false, "case": "XADUNI", "id": 393065, "judge": 18883, "vote": "Concurring"},
 JudgeRuling Object: {"author": false, "case": "X4A8KQ", "id": 393066, "judge": 19968, "vote": "Concurring"},
 JudgeRuling Object: {"author": false, "case": "XFJUJA", "id": 393067, "judge": 19378, "vote": "Concurring"},
 JudgeRuling Object: {"author": false, "case": "XFPV6I", "id": 393068, "judge": 20653, "vote": "Concurring"},
 JudgeRuling Object: {"author": false, "case": "XFHKC7", "id": 393069, "judge": 19142, "vote": "Concurring"},
 JudgeRuling Object: {"author": false, "case": "X320U8", "id": 393070, "judge": 19391, "vote": "Concurring"},
 JudgeRulin